In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pytpc
from pytpc.constants import *
import pytpc.simulation
import pytpc.tpcplot
import pytpc.vmedata
import pandas as pd
import matplotlib.pyplot as plt
import seaborn.apionly as sns
import scipy.signal
import h5py
from scipy.interpolate import UnivariateSpline, interp1d
from scipy.optimize import differential_evolution, minimize, basinhopping, leastsq
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import os
import cmaps
import yaml
from scipy.fftpack import fft, ifft, fftshift, ifftshift
from copy import copy, deepcopy

In [ ]:
sns.set_context('notebook')

In [ ]:
from pytpc.utilities import find_exclusion_region, read_lookup_table

In [ ]:
runtables = pd.read_csv('/Users/josh/Documents/Data/Meta/e15503b/e15503b-rundata-parseable.csv', index_col=0)

In [ ]:
from pytpc.relativity import find_proton_params

In [ ]:
lookup = read_lookup_table('/Users/josh/Documents/Data/Meta/e15503b/Lookup20150611.csv')
reverse_lookup = {v: k for k, v in lookup.items()}
excl_pads, lowg_pads = find_exclusion_region('/Users/josh/Documents/Data/Meta/e15503b/configs/run_0066/configure-e15503b.xcfg',
                                  lookup)

In [ ]:
vmegood = pd.read_hdf('/Users/josh/Documents/Data/Results/e15503b/select_vme/vme_good_events_e15503b.h5')

In [ ]:
def transfer(amp, shape, offset=0):
    t = np.arange(512)
    f = amp*np.exp(-3*(t-offset)/shape)*np.sin((t-offset)/shape)*((t-offset)/shape)**3 / 0.044
    return np.where(t >= offset, f, np.zeros(t.shape))

In [ ]:
def find_vertex_energy(beam_intercept):
    ei = 4.6 * 46 - 20.
    ri = gas.range(ei, 46, 18)  # this is in meters
    rf = ri - (1000 - beam_intercept) / 1000
    ef = gas.inverse_range(rf, 46, 18)
    return ef

def find_kine_vert_en(mproj, mtarg, scat_ang, recoil_ke):
    E_recoil = recoil_ke + mtarg
    cm_ang = np.pi - 2*scat_ang
    p_sq = (mtarg * E_recoil - mtarg**2) / (1 - np.cos(cm_ang))
    sqrts = np.sqrt(mtarg**2 + p_sq) + np.sqrt(mproj**2 + p_sq)
    T = 1 / (2 * mtarg) * (sqrts**2 - (mtarg + mproj)**2)
    return T

# Pad sizes

In [ ]:
pads = pytpc.generate_pad_plane()

In [ ]:
bigpads = np.where(np.round(np.abs(pads[:, 1, 1] - pads[:, 0, 1])) > 6)[0]
smallpads = np.where(np.round(np.abs(pads[:, 1, 1] - pads[:, 0, 1])) < 6)[0]
assert(len(bigpads) + len(smallpads) == 10240)

In [ ]:
evtlist = vmegood[vmegood.run_num == 193].evt_id.values

# Read event from results file

In [ ]:
run_num = 99
evt_id = 44  # good ones in run 99: 30, 44 (clean), 58, 60, 64, 73, 77 (bad), 81 (hard), 85, 284

In [ ]:
with pytpc.HDFDataFile('/Users/josh/Documents/Data/Merged/e15503b/full-trace/run_{:04d}.h5'.format(run_num), 'r') as f:
# with pytpc.HDFDataFile('/Volumes/attpc/data/e15503b/full-trace/run_{:04d}.h5'.format(run_num), 'r') as f: 
    evt = f[evt_id]

vme_offset = 0
with h5py.File('/Users/josh/Documents/Data/Merged/e15503b/VME/run_{}_vme.h5'.format(run_num), 'r') as f:
    icsignal = f['/vme/ic'][evt_id + vme_offset]
    trigsignal = f['/vme/trig'][evt_id + vme_offset]
    meshsignal = f['/vme/mesh'][evt_id + vme_offset]

In [ ]:
with open('../fitters/config_e15503b.yml', 'r') as f:
    config = yaml.load(f)
with open('../fitters/config_e15503b_macmini.yml', 'r') as f:
    config_patch = yaml.load(f)
config.update(config_patch)

# config['vdmag'] = -5.2
# config['tilt'] = 8

In [ ]:
# tmat = pytpc.utilities.tilt_matrix(-config['tilt']*degrees)
# emag = 9500
# bmag = 1.68

# config['efield'], config['bfield'] = map(lambda x: tmat @ x, 
#                                          pytpc.utilities.create_fields(emag, bmag, config['tilt']*degrees))
# config['vd'] = pytpc.simulation.drift_velocity_vector(config['vdmag'], emag, bmag, config['tilt']*degrees)
# config['vd'] = np.array([-0.053, -0.546, -5.25])

# config['gas_pressure'] = 19.2

In [ ]:
tilt = config['tilt'] * degrees # 6*degrees
clock = config['clock']
beam_en = config['beam_enu0']

efield = np.array(config['efield'])
bfield = np.array(config['bfield'])
tmat = pytpc.utilities.tilt_matrix(-tilt)
# efield_uvw = tmat.dot(efield)
# bfield_uvw = tmat.dot(bfield)

gas = pytpc.gases.InterpolatedGas('isobutane', config['gas_pressure'])

vd = np.array(config['vd'])
beampads = np.fromfile(config['beampads_path'], sep=',', dtype='int')

In [ ]:
efield

In [ ]:
bfield

In [ ]:
vd

In [ ]:
with h5py.File('/Users/josh/Documents/Data/Merged/e15503b/clean/clean_run_{:04d}.h5'.format(run_num), 'r') as h5file:
# with h5py.File('/Users/josh/Documents/Code/ar40-aug15/fitters/test.h5', 'r') as h5file:
    ds = h5file['/clean/{}'.format(evt_id)]
    raw_xyz_full = ds[:]
    cx_orig, cy_orig = ds.attrs['center'][:2]

In [ ]:
raw_xyz = pytpc.cleaning.apply_clean_cut(raw_xyz_full)
# raw_xyz = pytpc.cleaning.apply_clean_cut(raw_xyz_full, qthresh=0.5, nthresh=2, tbthresh=500)
center_data = raw_xyz[np.argsort(raw_xyz[:, 2])][-len(raw_xyz) // 2:]
# cx, cy = pytpc.cleaning.hough_circle(center_data)
cx, cy = cx_orig, cy_orig 

In [ ]:
plt.plot(raw_xyz_full[:, 2], raw_xyz_full[:, 1], 'r.')
plt.plot(raw_xyz[:, 2], raw_xyz[:, 1], 'b.')
plt.hlines(cy, *plt.xlim(), edgecolor='k')

In [ ]:
plt.plot(raw_xyz_full[:, 0], raw_xyz_full[:, 1], 'r.')
plt.plot(raw_xyz[:, 0], raw_xyz[:, 1], 'b.')
plt.plot(cx, cy, 'k+', mew=1.5, ms=10)
# plt.plot(testctr[0], testctr[1], 'gx', mew=1.5, ms=10)
plt.gca().set_aspect(1)

In [ ]:
hits = np.zeros(10240)
hits[raw_xyz[:, 4].astype('int')] = raw_xyz[:, 3]
hits[beampads] = 0
# pytpc.tpcplot.pad_plot(hits, scale='linear', cmap=cmaps.viridis)
pytpc.tpcplot.pad_plot(hits)
# plt.savefig('/Users/josh/Desktop/Figures/hits_{}-{}.pdf'.format(run_num, evt_id), transparent=True)

In [ ]:
plt.hist(hits[hits.nonzero()], bins=np.arange(0, 4096, 32), histtype='stepfilled');

In [ ]:
hitpads = raw_xyz[:, -1]

In [ ]:
peds = np.round(evt.traces['data'][:, 450:500].mean(1)).astype('int16')
evt.traces['data'] = (evt.traces['data'].T - peds).T

In [ ]:
padfilter = np.where(np.in1d(evt.traces['pad'], hitpads))[0]

In [ ]:
tr = evt.traces['data']
padlist = evt.traces[padfilter]['pad']

In [ ]:
fig, ax = plt.subplots(2, 1, squeeze=True, sharex=True)

xs = np.arange(-101, 411)

ax[0].plot(xs, icsignal)
ax[0].plot(xs, trigsignal)
ax[0].plot(xs, meshsignal)

ax[0].axvspan(0-100, 152-100, facecolor='gray', alpha=0.2)
ax[0].axvspan(392-100, 511-100, facecolor='gray', alpha=0.2)
ax[0].axvspan(411, 511, facecolor='gray', alpha=0.5, hatch='/')
ax[0].set_xlim(-100, 511)

ax[1].plot(tr.T);

ax[1].axvspan(-100, 0, facecolor='gray', alpha=0.5, hatch='/')
ax[1].axvspan(0, 52, facecolor='gray', alpha=0.2)
ax[1].axvspan(292, 511, facecolor='gray', alpha=0.2)
# ax[1].plot(raw_xyz[:, 2], raw_xyz[:, 3], 'rv')
ax[1].set_xlim(-100, 511)

In [ ]:
plt.plot(evt.traces[padfilter]['data'].T);
plt.xlim(45, 250)

In [ ]:
plt.hist(raw_xyz[:, 3], bins=np.arange(0, 4096, 256));

In [ ]:
i = -1

In [ ]:
i -= 1

In [ ]:
# i += 1
p = raw_xyz[i, -1]
idx = np.where(evt.traces['pad'] == p)[0]
v = evt.traces[idx]['data'].ravel()
# plt.plot(raw_xyz[i, 2], raw_xyz[i, 3], 'rv')
plt.plot(v)
sns.despine()
plt.xlim(0, 511)
plt.xlabel('ADC time bucket')
plt.ylabel('ADC signal amplitude')
# plt.savefig('/Users/josh/Desktop/Figures/baseline_depression.pdf', transparent=True)

---
# MC Minimizer

In [ ]:
mcfitter = pytpc.fitting.MCFitter(config)

In [ ]:
# mcfitter.beampads = np.array([])

In [ ]:
# mcfitter.minimizer.enChi2NormFraction = 0.1
# mcfitter.minimizer.vertChi2Enabled = False
# mcfitter.minimizer.enChi2Enabled = True
# mcfitter.minimizer.posChi2Enabled = False
# mcfitter.sigma = np.array([0.1, 0.1, 0.05, 2.0, 10*degrees, 10*degrees])

In [ ]:
mcfitter.sigma

In [ ]:
mcfitter.vd

In [ ]:
xyz, (cu, cv) = mcfitter.preprocess(raw_xyz, center=(cx, cy))
mcres, minChis, all_params, good_param_idx = mcfitter.process_event(xyz, cu, cv, return_details=True)
# print(mcres['enChi2'])
# print(minChis[-1] - minChis[0])

In [ ]:
res_track = mcfitter.tracker.track_particle(mcres['x0'], mcres['y0'], mcres['z0'], 
                                            mcres['enu0'], mcres['azi0'], mcres['pol0'])
xyz = mcfitter.preprocess(raw_xyz)

In [ ]:
good_params = all_params[good_param_idx.astype('int')]
all_params = all_params.reshape((mcfitter.num_iters, mcfitter.num_pts, 6))

In [ ]:
lines = plt.plot(minChis, 'o-')
plt.plot(minChis.sum(1))
plt.semilogy()
plt.legend(lines, ('pos', 'en', 'vert'))

In [ ]:
for v, l in zip(good_params.T, ['x', 'y', 'z', 'en', 'azi', 'pol', 'bmag']):
    plt.plot(v / v[0], label=l)
plt.legend(loc=4)

In [ ]:
fig, ax = plt.subplots(3, 2, sharex=True)

plt.sca(ax[0, 0])
plt.plot(np.ravel(all_params[:, :, 0]), ',')
plt.text(0.95, 0.9, 'x', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
plt.sca(ax[1, 0])
plt.plot(np.ravel(all_params[:, :, 1]), ',')
plt.text(0.95, 0.9, 'y', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
plt.sca(ax[2, 0])
plt.plot(np.ravel(all_params[:, :, 2]), ',')
plt.text(0.95, 0.9, 'z', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
# plt.hlines(zi, *plt.xlim())
plt.sca(ax[0, 1])
plt.plot(np.ravel(all_params[:, :, 3]), ',')
plt.text(0.95, 0.9, 'E', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
# plt.hlines(eni, *plt.xlim())
plt.sca(ax[1, 1])
plt.plot(np.ravel(all_params[:, :, 4]), ',')
plt.text(0.95, 0.9, 'φ', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
# plt.hlines(azii, *plt.xlim())
plt.sca(ax[2, 1])
plt.plot(np.ravel(all_params[:, :, 5]), ',')
plt.text(0.95, 0.9, 'θ', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})

# plt.hlines(poli, *plt.xlim())

sns.despine()

plt.tight_layout()
# plt.savefig('/Users/josh/Desktop/yassid poster/mcsamples.pdf', transparent=True)

In [ ]:
with sns.axes_style(rc={'font.family': 'Arial'}), sns.plotting_context('paper', rc={'font.size': 10}):
    plt.figure(figsize=(3.5, 2.5))
    plt.plot(all_params[:, :, 3].ravel(), ',')
    plt.xticks(np.arange(0, mcfitter.num_pts * (mcfitter.num_iters + 1), mcfitter.num_pts * 5),
               np.arange(0, mcfitter.num_iters + 1, 5))
    sns.despine()
    
    plt.xlabel('Iteration number')
    plt.ylabel('Proton energy [MeV]')
    
    plt.tight_layout(pad=0.5)
    plt.subplots_adjust(right=0.9)
    
    plt.xlim(0)
    plt.ylim(0)
    
    plt.savefig('/Users/josh/Documents/Papers/attpc-nim-paper/Figures/mcsamples.pdf')

In [ ]:
# ###### HAND FIT #######################

# mcres = {'azi0': 140 * degrees,
#  'brho': 0.11715228451358371,
#  'curv_ctr_x': -2.511620781172536,
#  'curv_ctr_y': 40.72253611548619,
#  'curv_en': 0.6573310390311133,
#  'enChi2': 1.4024581524902917,
#  'enu0': 0.9,
#  'lin_beam_int': 296.8827014256685,
#  'lin_chi2': 5.8181296682890755,
#  'lin_scat_ang': 1.0071015791003997,
#  'pol0': 2.3,
#  'posChi2': 3.928712700799101,
#  'rad_curv': 58.94480470133671,
#  'vertChi2': 2.615874479899877,
#  'x0': -0.04,
#  'y0': -0.0,
#  'z0': 0.27}

# res_track = mcfitter.tracker.track_particle(mcres['x0'], mcres['y0'], mcres['z0'], 
#                                             mcres['enu0'], mcres['azi0'], mcres['pol0'])
# xyz = mcfitter.preprocess(raw_xyz)

In [ ]:
kine_vert_en = find_kine_vert_en(46*p_mc2, 1*p_mc2, pi - mcres['pol0'], mcres['enu0']) / 46
vert_en = find_vertex_energy(mcres['z0'] * 1000) / 46
print('Kin vert en:', kine_vert_en)
print('Pos vert en:', vert_en)
print('      Delta:', kine_vert_en - vert_en)

In [ ]:
plt.plot(xyz.u / 1000, xyz.v / 1000, '.')
plt.plot(res_track[:, 0], res_track[:, 1])
plt.gca().set_aspect(1)
# plt.axis([-0.275, 0.275, -0.275, 0.275])
# ths = np.linspace(0, 2*pi, 100)
# plt.plot(0.275 * np.cos(ths), 0.275 * np.sin(ths), 'r--', linewidth=1)

In [ ]:
mcres

In [ ]:
m = 1# 0.85
b = 0# 0.05

plt.plot(xyz.w / 1000 * m + b, xyz.v / 1000, 'b.')
plt.plot(res_track[:, 2], res_track[:, 1], 'g')

In [ ]:
with sns.plotting_context('talk', font_scale=1.2):
    plt.plot(xyz.z, xyz.y, '.')
    plt.xlabel('x [mm]')
    plt.ylabel('y [mm]')
#     plt.savefig('/Users/josh/Desktop/MC problems/tilt.pdf')

In [ ]:
len(xyz)

In [ ]:
len(raw_xyz_full)

In [ ]:
plt.plot(raw_xyz_full[:, 2], raw_xyz_full[:, 1], '.')

In [ ]:
plt.plot(xyz.w / 1000 * m + b, xyz.u / 1000, 'r.')
plt.plot(res_track[:, 2], res_track[:, 0], 'm')
# plt.gca().set_aspect(1)

In [ ]:
plt.plot(xyz.w / 1000, np.hypot(xyz.u, xyz.v) / 1000, 'b.')
plt.plot(res_track[:, 2], np.hypot(res_track[:, 0], res_track[:, 1]), 'g-')

In [ ]:
tr_min = res_track[:, 2].min()
tr_max = res_track[:, 2].max()
datacut = (xyz.w > tr_min * 1000) & (xyz.w < tr_max * 1000)
print(tr_max * 1000 - xyz.loc[datacut, 'w'].max())

hbins = np.arange(tr_min, tr_max, abs(vd[2]) / clock / 100)
bins, *junk = np.histogram(xyz.loc[datacut, 'w'] / 1000, bins=hbins)

filledbins = len(bins.nonzero()[0])
emptybins = len(bins) - filledbins
print(emptybins, filledbins, len(bins))
print(emptybins / len(bins), filledbins / len(bins))

In [ ]:
good_param_idx

In [ ]:
sns.set_context('talk', font_scale=1.6)

In [ ]:
for p in good_params:
    tr = mcfitter.tracker.track_particle(*p)
    rej_tr_line, = plt.plot(tr[:, 2] * 1000, tr[:, 1] * 1000, 'b-', alpha=0.5, zorder=100)
best_line, = plt.plot(res_track[:, 2] * 1000, res_track[:, 1] * 1000, 'k-', linewidth=3, zorder=300)
exp_line, = plt.plot(xyz.w, xyz.v, 'r.', zorder=200)
sns.despine()
plt.legend((rej_tr_line, best_line, exp_line), ('Rejected track', 'Best fit track', 'Experimental data'), loc=9, 
           frameon=False, numpoints=3, ncol=2)
plt.ylim(-50, 300)
plt.xlim(100, 725)
plt.xlabel('z [mm]')
plt.ylabel('y [mm]')
# plt.savefig('/Users/josh/Desktop/Figures/fitplot.pdf', transparent=True)

In [ ]:
all_params.shape

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(all_params[:, :, 2].ravel() * 1000, '.', markersize=4);
plt.xlabel('Iteration index')
plt.ylabel('z0 [mm]')
sns.despine()
# plt.savefig('/Users/josh/Desktop/Figures/z0_fit_conv.png', transparent=True, dpi=200, bbox_inches='tight')

In [ ]:
mcres

In [ ]:
# %matplotlib notebook

In [ ]:
with sns.axes_style('white'):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(*res_track[:, :3].T, 'k')
    ax.scatter(xyz.u / 1000, xyz.v / 1000, xyz.w / 1000)
    ax.set_aspect(1)
    
#     plt.savefig('/Users/josh/Desktop/yassid poster/3d.pdf', transparent=True)

In [ ]:
hits2 = hits.copy()
hits2[lowg_pads] += 100
pytpc.tpcplot.pad_plot(hits2)
# plt.savefig('/Users/josh/Desktop/Figures/hits_{}-{}.pdf'.format(run_num, evt_id), transparent=True)

In [ ]:
hits_xyz = mcfitter.evtgen.make_hit_pattern(xyz[['u', 'v', 'w']].values / 1000., xyz.a.values)
hits_xyz -= hits * hits_xyz.max() / hits.max()

lim = np.abs(hits_xyz).max()
fig, sm = pytpc.tpcplot.pad_plot(hits_xyz, scale='linear', cmap='RdBu', cmin=-lim, cmax=lim)
plt.colorbar(sm)

In [ ]:
res_hits = mcfitter.evtgen.make_hit_pattern(res_track[:, :3], res_track[:, 4])
# res_hits[lowg_pads] += 100
pytpc.tpcplot.pad_plot(res_hits)

In [ ]:
hitdiff = mcfitter.minimizer.find_hit_pattern_deviation(res_track[:, :3], res_track[:, 4], hits)
scale_limit = abs(hitdiff).max() * 0.5
fig, sm = pytpc.tpcplot.pad_plot(hitdiff, scale='linear', cmap='RdBu', cmin=-scale_limit, cmax=scale_limit)
plt.colorbar(sm)

In [ ]:
sns.set_context('talk')

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
# fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax[0, 0].plot(xyz.w / 1000, xyz.u / 1000, '.')
ax[0, 0].plot(res_track[:, 2], res_track[:, 0], 'k')
ax[0, 0].set_xlabel('z [m]')
ax[0, 0].set_ylabel('x [m]')

ax[0, 1].plot(xyz.w / 1000, xyz.v / 1000, '.')
ax[0, 1].plot(res_track[:, 2], res_track[:, 1], 'k')
ax[0, 1].set_xlabel('z [m]')
ax[0, 1].set_ylabel('y [m]')

ax[1, 0].plot(xyz.w/1000, np.hypot(xyz.u, xyz.v) / 1000, '.')
ax[1, 0].plot(res_track[:, 2], np.hypot(res_track[:, 0], res_track[:, 1]), 'k')
ax[1, 0].set_xlabel('z [m]')
ax[1, 0].set_ylabel('$\sqrt{x^2 + y^2}$ [m]')

ax[1, 1].plot(xyz.u / 1000, xyz.v / 1000, '.')
ax[1, 1].plot(res_track[:, 0], res_track[:, 1], 'k')
ths = np.linspace(0, 2*pi, 100)
# ax[1, 1].plot(mcres['rad_curv']/1000 * np.cos(ths) + mcres['curv_ctr_x']/1000, 
#               mcres['rad_curv']/1000 * np.sin(ths) + mcres['curv_ctr_y']/1000, 'r--')
# ax[1, 1].plot(cu/1000, cv/1000, 'k+', mew=1.5)
ax[1, 1].plot(mcres['curv_ctr_x']/1000, mcres['curv_ctr_y']/1000, 'ro')
ax[1, 1].set_aspect(1)
ax[1, 1].set_xlabel('x [m]')
ax[1, 1].set_ylabel('y [m]')

sns.despine()

plt.tight_layout()
# plt.savefig('/Users/josh/Desktop/Figures/fit_{}-{}.pdf'.format(run_num, evt_id), transparent=True)

In [ ]:
sns.set_context('talk', font_scale=1.6)

In [ ]:
window_min = (-20 + 275) / (275 + 275)
window_max = (20 + 275) / (275 + 275)

plt.plot(xyz.w, xyz.v, '.')
plt.plot(res_track[:, 2] * 1000, res_track[:, 1] * 1000, 'k-')
plt.axvspan(-20, 0, edgecolor='k', facecolor='grey', linewidth=1, hatch='\\')
plt.axvspan(1000, 1020, ymin=0, ymax=window_min, edgecolor='k', facecolor='grey', linewidth=1, hatch='\\')
plt.axvspan(1000, 1020, ymin=window_min, ymax=window_max, edgecolor='k', facecolor='c', linewidth=1)
plt.axvspan(1000, 1020, ymin=window_max, ymax=1, edgecolor='k', facecolor='grey', linewidth=1, hatch='\\')
plt.axvspan(0, 1000, edgecolor='k', facecolor='grey', linewidth=1, alpha=0.2)
plt.hlines(0, 0, 1100, edgecolors='k', linewidth=3)
plt.xlim(-20, 1100)
plt.ylim(-275, 275)
sns.despine()

plt.xlabel('z [mm]')
plt.ylabel('y [mm]')
plt.gca().set_aspect(1)

# plt.savefig('/Users/josh/Desktop/Figures/projection_with_chamber.pdf', transparent=True)

In [ ]:
plt.plot(xyz.u, xyz.v, '.', zorder=150, label='Data')
plt.plot(res_track[:, 0] * 1000, res_track[:, 1] * 1000, 'k-', zorder=200, label='Fit result')
plt.plot(res_track[0, 0] * 1000, res_track[0, 1] * 1000, 'ro', markersize=15, zorder=100, label='Beam location')

plt.xlim(-200, 50)
plt.ylim(-60, 200)

plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

sns.despine()

plt.gca().set_aspect(1)

plt.savefig('/Users/josh/Desktop/Figures/projection_xy.pdf', transparent=True)

In [ ]:
res_evt = mcfitter.evtgen.make_event(res_track[:, :3], res_track[:, 4])

In [ ]:
0.020 * 500 * 20 / 28

In [ ]:
for pad, trace in res_evt.items():
    plt.plot(trace)

In [ ]:
plt.plot(evt.traces[padfilter]['data'].T);
# plt.xlim(45, 250)

In [ ]:
exp_pads = set(hits.nonzero()[0])
res_pads = set(res_evt.keys())
both_pads = exp_pads.intersection(res_pads)

In [ ]:
pad_iter = iter(both_pads)

In [ ]:
p = next(pad_iter)
plt.plot(evt.traces[np.where(evt.traces['pad'] == p)]['data'].ravel(), 'b')
plt.plot(np.roll(res_evt[p], 38), 'r')

In [ ]:
exp_hitpadnos, = hits.nonzero()
res_hitpadnos, = res_hits.nonzero()
bothhitct = len(np.intersect1d(exp_hitpadnos, res_hitpadnos))
missedct = len(hits.nonzero()[0]) - bothhitct
print('Both hit {} pads'.format(bothhitct))
print('Result missed {} pads'.format(missedct))

In [ ]:
plt.plot(xyz.w, xyz.pad, '.')
plt.plot(res_track)

# Annealer

In [ ]:
import annealmin

In [ ]:
# config['sigma']['azi'] = 60
# config['sigma']['x'] = 0.05
# config['sigma']['y'] = 0.1
config['sigma']['z'] = 0.
# config['sigma']['enu'] = 6.0
# config['diffusion_sigma'] = 0.1e-3
# config['num_iters'] = 30
# config['num_pts'] = 500
# config['red_factor'] = 0.90

config['anneal_num_iters'] = 30
config['cool_rate'] = 0.80
config['max_calls_per_iter'] = 2500
config['initial_temp'] = 0.5

In [ ]:
xs = np.arange(config['anneal_num_iters'])
plt.plot(xs, config['initial_temp'] * config['cool_rate'] ** xs)

In [ ]:
mcfitter = mcmin.MCFitter(config)
# anfitter = annealmin.Annealer(config)
# anfitter.sigma = np.array([0.05, 0.05, 0.005, 0.1, 5*degrees, 5*degrees])

In [ ]:
mcfitter.minimizer.enChi2NormFraction = 0.02
# mcfitter.minimizer.vertChi2Enabled = False
# anfitter.annealer.vertChi2Enabled = False
# mcfitter.minimizer.posChi2Enabled = True
# anfitter.annealer.posChi2Enabled = False
# mcfitter.sigma = np.array([0.1, 0.1, 0.05, 2.0, 10*degrees, 10*degrees])

In [ ]:
mcfitter.sigma

In [ ]:
mcfitter.vd

In [ ]:
anfitter.sigma * config['cool_rate'] ** config['anneal_num_iters']

In [ ]:
config['initial_temp'] * config['cool_rate'] ** config['anneal_num_iters']

In [ ]:
mcres, minChis, all_params, good_param_idx = mcfitter.process_event(raw_xyz, cx, cy, return_details=True)
print(mcres['enChi2'])
print(minChis[-1] - minChis[0])

In [ ]:
anfitter.annealer.initial_temp = 0.01

In [ ]:
anres, annealInfo = anfitter.process_event(raw_xyz, cx, cy, multi=False, return_details=True)

In [ ]:
annealInfo['stop_reason'], annealInfo['num_calls']

In [ ]:
plt.plot(annealInfo['chis'])
plt.plot(annealInfo['chis'].sum(1))
plt.semilogy()

In [ ]:
annctr = annealInfo['ctrs']

In [ ]:
plt.plot(annctr[:, 2], annctr[:, 5])
plt.scatter(annctr[:, 2], annctr[:, 5], c=annealInfo['chis'].sum(1), s=50)
plt.colorbar()

In [ ]:
res_track = mcfitter.tracker.track_particle(mcres['x0'], mcres['y0'], mcres['z0'], 
                                            mcres['enu0'], mcres['azi0'], mcres['pol0'])
xyz = mcfitter.preprocess(raw_xyz)

In [ ]:
res_track = mcfitter.tracker.track_particle(anres['x0'], anres['y0'], anres['z0'], 
                                            anres['enu0'], anres['azi0'], anres['pol0'])
xyz = mcfitter.preprocess(raw_xyz)

In [ ]:
good_param_idx

In [ ]:
good_params = all_params[good_param_idx.astype('int')]
all_params = all_params.reshape((mcfitter.num_iters, mcfitter.num_pts, 6))

In [ ]:
lines = plt.plot(minChis, 'o-')
plt.plot(minChis.sum(1))
plt.semilogy()
plt.legend(lines, ('pos', 'en', 'vert'))

In [ ]:
for v, l in zip(good_params.T, ['x', 'y', 'z', 'en', 'azi', 'pol', 'bmag']):
    plt.plot(v / v[0], label=l)
plt.legend(loc=4)

In [ ]:
fig, ax = plt.subplots(3, 2, sharex=True)

plt.sca(ax[0, 0])
plt.plot(np.ravel(all_params[:, :, 0]), ',')
plt.text(0.95, 0.9, 'x', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
plt.sca(ax[1, 0])
plt.plot(np.ravel(all_params[:, :, 1]), ',')
plt.text(0.95, 0.9, 'y', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
plt.sca(ax[2, 0])
plt.plot(np.ravel(all_params[:, :, 2]), ',')
plt.text(0.95, 0.9, 'z', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
# plt.hlines(zi, *plt.xlim())
plt.sca(ax[0, 1])
plt.plot(np.ravel(all_params[:, :, 3]), ',')
plt.text(0.95, 0.9, 'E', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
# plt.hlines(eni, *plt.xlim())
plt.sca(ax[1, 1])
plt.plot(np.ravel(all_params[:, :, 4]), ',')
plt.text(0.95, 0.9, 'φ', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})
# plt.hlines(azii, *plt.xlim())
plt.sca(ax[2, 1])
plt.plot(np.ravel(all_params[:, :, 5]), ',')
plt.text(0.95, 0.9, 'θ', transform=plt.gca().transAxes, ha='right', va='top', fontdict={'fontsize': 18})

# plt.hlines(poli, *plt.xlim())

sns.despine()

plt.tight_layout()
# plt.savefig('/Users/josh/Desktop/yassid poster/mcsamples.pdf', transparent=True)

In [ ]:
mcres

In [ ]:
# ###### HAND FIT #######################

# mcres = {'azi0': -2.9030341276714813,
#  'brho': 0.1925244050145264,
#  'curv_ctr_x': -29.376684318652973,
#  'curv_ctr_y': 131.77785709153548,
#  'curv_en': 1.7752282320983397,
#  'enChi2': 1.715910334525202,
#  'enu0': 2.1282424755195235,
#  'lin_beam_int': 602.1791411838143,
#  'lin_chi2': 8.746921435794643,
#  'lin_scat_ang': 1.152373719692896,
#  'pol0': 2.010,
#  'posChi2': 0.0,
#  'rad_curv': 106.61546521529228,
#  'vertChi2': 0.0,
#  'x0': 0.005477232206347699,
#  'y0': 0.024059177731298485,
#  'z0': 0.613}

# res_track = mcfitter.tracker.track_particle(mcres['x0'], mcres['y0'], mcres['z0'], 
#                                             mcres['enu0'], mcres['azi0'], mcres['pol0'])
# xyz = mcfitter.preprocess(raw_xyz)

In [ ]:
kine_vert_en = find_kine_vert_en(46*p_mc2, 1*p_mc2, pi - mcres['pol0'], mcres['enu0']) / 46
vert_en = find_vertex_energy(mcres['z0'] * 1000) / 46
print('Kin vert en:', kine_vert_en)
print('Pos vert en:', vert_en)
print('      Delta:', kine_vert_en - vert_en)

In [ ]:
plt.plot(xyz.u / 1000, xyz.v / 1000, '.')
plt.plot(res_track[:, 0], res_track[:, 1])
plt.gca().set_aspect(1)
# plt.axis([-0.275, 0.275, -0.275, 0.275])
# ths = np.linspace(0, 2*pi, 100)
# plt.plot(0.275 * np.cos(ths), 0.275 * np.sin(ths), 'r--', linewidth=1)

In [ ]:
m = 1# 0.85
b = 0# 0.05

plt.plot(xyz.w / 1000 * m + b, xyz.v / 1000, 'b.')
plt.plot(res_track[:, 2], res_track[:, 1], 'g')

plt.plot(xyz.w / 1000 * m + b, xyz.u / 1000, 'r.')
plt.plot(res_track[:, 2], res_track[:, 0], 'm')
# plt.gca().set_aspect(1)

In [ ]:
plt.plot(xyz.w / 1000, np.hypot(xyz.u, xyz.v) / 1000, 'b.')
plt.plot(res_track[:, 2], np.hypot(res_track[:, 0], res_track[:, 1]), 'g-')

In [ ]:
mcres

In [ ]:
(pi-mcres['pol0']) / degrees

In [ ]:
# %matplotlib notebook

In [ ]:
with sns.axes_style('white'):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(*res_track[:, :3].T, 'k')
    ax.scatter(xyz.u / 1000, xyz.v / 1000, xyz.w / 1000)
    ax.set_aspect(1)
    
#     plt.savefig('/Users/josh/Desktop/yassid poster/3d.pdf', transparent=True)

In [ ]:
sns.set_style('ticks')

In [ ]:
### fig, ax = plt.subplots(2, 2, figsize=(12, 10))
# fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax[0, 0].plot(xyz.w / 1000, xyz.u / 1000, '.')
ax[0, 0].plot(res_track[:, 2], res_track[:, 0], 'k')
ax[0, 0].set_xlabel('z [m]')
ax[0, 0].set_ylabel('x [m]')

ax[0, 1].plot(xyz.w / 1000, xyz.v / 1000, '.')
ax[0, 1].plot(res_track[:, 2], res_track[:, 1], 'k')
ax[0, 1].set_xlabel('z [m]')
ax[0, 1].set_ylabel('y [m]')

ax[1, 0].plot(xyz.w/1000, np.hypot(xyz.u, xyz.v) / 1000, '.')
ax[1, 0].plot(res_track[:, 2], np.hypot(res_track[:, 0], res_track[:, 1]), 'k')
ax[1, 0].set_xlabel('z [m]')
ax[1, 0].set_ylabel('$\sqrt{x^2 + y^2}$ [m]')

ax[1, 1].plot(xyz.u / 1000, xyz.v / 1000, '.')
ax[1, 1].plot(res_track[:, 0], res_track[:, 1], 'k')
ths = np.linspace(0, 2*pi, 100)
# ax[1, 1].plot(mcres['rad_curv']/1000 * np.cos(ths) + mcres['curv_ctr_x']/1000, 
#               mcres['rad_curv']/1000 * np.sin(ths) + mcres['curv_ctr_y']/1000, 'r--')
# ax[1, 1].plot(cu/1000, cv/1000, 'k+', mew=1.5)
ax[1, 1].plot(mcres['curv_ctr_x']/1000, mcres['curv_ctr_y']/1000, 'ro')
ax[1, 1].set_aspect(1)
ax[1, 1].set_xlabel('x [m]')
ax[1, 1].set_ylabel('y [m]')

sns.despine()

plt.tight_layout()
# plt.savefig('/Users/josh/Desktop/Figures/fit_{}-{}.pdf'.format(run_num, evt_id), transparent=True)

In [ ]:
hits.max()

In [ ]:
hits2 = hits.copy()
hits2[lowg_pads] += 100
pytpc.tpcplot.pad_plot(hits2)
# plt.savefig('/Users/josh/Desktop/Figures/hits_{}-{}.pdf'.format(run_num, evt_id), transparent=True)

In [ ]:
res_hits = mcfitter.evtgen.make_hit_pattern(res_track[:, :3], res_track[:, 4])
res_hits[lowg_pads] += 100
pytpc.tpcplot.pad_plot(res_hits)

In [ ]:
hitdiff = mcfitter.minimizer.find_hit_pattern_deviation(res_track[:, :3], res_track[:, 4], hits)
scale_limit = abs(hitdiff).max() * 0.1
fig, sm = pytpc.tpcplot.pad_plot(hitdiff, scale='linear', cmap='RdBu', cmin=-scale_limit, cmax=scale_limit)
plt.colorbar(sm)

In [ ]:
res_track2 = mcfitter.tracker.track_particle(mcres['x0'], mcres['y0'], mcres['z0'], 
                                            mcres['enu0'], mcres['azi0'], mcres['pol0'])

hitdiff = mcfitter.minimizer.find_hit_pattern_deviation(res_track2[:, :3], res_track2[:, 4], hits)
scale_limit = abs(hitdiff).max() * 0.1
fig, sm = pytpc.tpcplot.pad_plot(hitdiff, scale='linear', cmap='RdBu', cmin=-scale_limit, cmax=scale_limit,
                                 pads=pytpc.generate_pad_plane(-108*degrees))
plt.colorbar(sm)

In [ ]:
vd

# Convergence testing

In [ ]:
# anfitter.sigma = np.array([0.01, 0.01, 0.01, 0.05, 5*degrees, 5*degrees])
anfitter.annealer.num_iters = 100
anfitter.annealer.initial_temp = 2.0
anfitter.annealer.cool_rate = 0.95
# anfitter.annealer.max_calls_per_iter = 2000
# anfitter.annealer.posChi2Enabled = True
# anfitter.annealer.enChi2Enabled = True
# anfitter.annealer.vertChi2Enabled = True

In [ ]:
all_mcres = []
for i in range(5):
    mcres, minChis, all_params, good_param_idx = mcfitter.process_event(raw_xyz, cx, cy, return_details=True)
#     mcres = anfitter.process_event(raw_xyz, cx, cy)
    all_mcres.append(mcres)

In [ ]:
all_mcres_multi = []
all_aninfo = []
for i in range(10):
#     mcres, minChis, all_params, good_param_idx = mcfitter.process_event(raw_xyz, cx, cy, return_details=True)
    mcres, aninfo = anfitter.process_event(raw_xyz, cx, cy, multi=False, return_details=True)
    all_mcres_multi.append(mcres)
    all_aninfo.append(aninfo)

In [ ]:
all_mcres = pd.DataFrame(all_mcres)
all_mcres['totChi2'] = all_mcres.posChi2 + all_mcres.enChi2 + all_mcres.vertChi2

In [ ]:
all_mcres_multi = pd.DataFrame(all_mcres_multi)
all_mcres_multi['totChi2'] = all_mcres_multi.posChi2 + all_mcres_multi.enChi2 + all_mcres_multi.vertChi2

In [ ]:
best = all_mcres.totChi2.argmin()
all_mcres.loc[best]

In [ ]:
best1 = all_mcres_multi.totChi2.argmin()
all_mcres_multi.loc[best]

In [ ]:
all_mcres_multi.loc[best1] - all_mcres.loc[best]

In [ ]:
all_mcres.max() - all_mcres.min()

In [ ]:
diff = all_mcres_multi.max() - all_mcres_multi.min()
diff[['x0', 'y0', 'z0', 'enu0', 'azi0', 'pol0']]

In [ ]:
best = np.argmin(all_mcres.totChi2)
for i, mcres in all_mcres.iterrows():
    res_track = mcfitter.tracker.track_particle(mcres['x0'], mcres['y0'], mcres['z0'], 
                                                mcres['enu0'], mcres['azi0'], mcres['pol0'])
    if i == best:
        props = dict(linewidth=2, alpha=1.0, color='k')
    else:
        props = dict(linewidth=1, alpha=0.2, color='b')
    plt.plot(res_track[:, 0] * 1000, res_track[:, 1] * 1000, **props)
plt.plot(xyz.u, xyz.v, '.')
plt.gca().set_aspect(1)

In [ ]:
best = np.argmin(all_mcres_multi.totChi2)
for i, mcres in all_mcres_multi.iterrows():
    res_track = mcfitter.tracker.track_particle(mcres['x0'], mcres['y0'], mcres['z0'], 
                                                mcres['enu0'], mcres['azi0'], mcres['pol0'])
    if i == best:
        props = dict(linewidth=2, alpha=1.0, color='k')
    else:
        props = dict(linewidth=1, alpha=0.2, color='b')
    plt.plot(res_track[:, 0] * 1000, res_track[:, 1] * 1000, **props)
plt.plot(xyz.u, xyz.v, '.')
plt.gca().set_aspect(1)

In [ ]:
best = np.argmin(all_mcres_multi.totChi2)
for i, mcres in all_mcres_multi.iterrows():
    res_track = mcfitter.tracker.track_particle(mcres['x0'], mcres['y0'], mcres['z0'], 
                                                mcres['enu0'], mcres['azi0'], mcres['pol0'])
    if i == best:
        props = dict(linewidth=2, alpha=1.0, color='k')
    else:
        props = dict(linewidth=1, alpha=0.2, color='b')
    plt.plot(res_track[:, 2] * 1000, res_track[:, 1] * 1000, **props)
plt.plot(xyz.w, xyz.v, '.')
plt.gca().set_aspect(1)

In [ ]:
anfitter.sigma[[3, 5]]

In [ ]:
v1 = 3
v2 = 5
for a in all_aninfo:
    c = a['ctrs']
    x = a['chis']
    plt.plot(c[:, v1], c[:, v2])
    plt.scatter(c[:, v1], c[:, v2], c=x.sum(1), s=50, zorder=100)
    plt.scatter(c[-1, v1], c[-1, v2], c=x.sum(1)[-1], s=50, linewidth=1.5, zorder=200)
    plt.scatter(c[0, v1], c[0, v2], c=x.sum(1)[-1], s=50, linewidth=1.5, edgecolor='r', zorder=200)
plt.colorbar()
# plt.xlim(2.0, 2.2)
# plt.ylim(2.0, 2.1)

In [ ]:
with sns.axes_style('white'):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    v1 = 3
    v2 = 5
    v3 = 4
    for a in all_aninfo:
        c = a['ctrs']
        x = a['chis']
        ax.plot(c[:, v1], c[:, v2], c[:, v3])
        ax.scatter(c[:, v1], c[:, v2], c[:, v3], c=x.sum(1), s=50, zorder=100)
#         ax.scatter(c[-1, v1], c[-1, v2], c[-1, v3], c=x.sum(1)[-1], s=50, linewidth=1.5, zorder=200)
#         ax.scatter(c[0, v1], c[0, v2], c[0, v3], c=x.sum(1)[-1], s=50, linewidth=1.5, edgecolor='r', zorder=200)

In [ ]:
sns.boxplot(y='z0', data=all_mcres)

In [ ]:
plt.plot(all_mcres.enu0, 'o')

# Test on other functions

In [ ]:
def sphere(p):
    return np.sum(p**2)
def rosen2d(p, a, b):
    return (a - p[0])**2 + b * (p[1] - p[0]**2)**2
def egg_crate(p):
    return -(p[1]+47)*np.sin(np.sqrt(np.abs(p[1]+p[0]/2+47))) - p[1]*np.sin(np.sqrt(np.abs(p[0]-(p[1]+47))))

In [ ]:
guess = np.array([-3.0, 3.0])
delta = np.full(2, 1.0)

# ctr, min_chis, good_params, all_params = montecarlo.mcmin(sphere, guess, delta, 
#                                                           print_status=True, detailed_output=True,
#                                                           red_factor=0.80, num_iters=10, num_pts=100)
# good_params, min_chis = metro(sphere, guess, delta, T0=10, cool_rate=0.5, num_iters=20)
good_params = [guess]
optres = minimize(sphere, guess, callback=good_params.append)
print(optres)
good_params = np.array(good_params)
# print(good_params[-1])
# plt.plot(min_chis)

In [ ]:
X, Y = np.meshgrid(np.linspace(-5, 5, 100), np.linspace(-5, 5, 100))

XY = np.column_stack((X.ravel(), Y.ravel()))

Z = np.apply_along_axis(sphere, axis=1, arr=XY)
Z = Z.reshape(X.shape)

plt.plot(good_params[:, 0], good_params[:, 1], 'ro-')
plt.contour(X, Y, Z, levels=np.linspace(0, 50, 20), colors='k', linewidths=1)
plt.gca().set_aspect(1)

In [ ]:
guess = np.array([-0.5, -1.5])
delta = np.full(2, 0.75)

# ctr, min_chis, good_params, all_params = montecarlo.mcmin(rosen2d, guess, delta, func_kwargs={'a': 1, 'b': 100},
#                                                           print_status=True, detailed_output=True,
#                                                           red_factor=0.90, num_iters=50, num_pts=500)

# good_params, min_chis = metro(rosen2d, guess, delta, T0=4, cool_rate=0.50, num_iters=20, max_calls=10000,
#                               func_kwargs={'a': 1, 'b': 100})

good_params = [guess]
optres = minimize(rosen2d, guess, args=(1, 100),callback=good_params.append)
print(optres)
good_params = np.array(good_params)

# print(good_params[-1])
# plt.plot(min_chis)

In [ ]:
X, Y = np.meshgrid(np.linspace(-2, 2, 100), np.linspace(-2, 2, 100))

XY = np.column_stack((X.ravel(), Y.ravel()))

Z = np.apply_along_axis(rosen2d, axis=1, arr=XY, a=1, b=100)
Z = Z.reshape(X.shape)

plt.plot(good_params[:, 0], good_params[:, 1], 'r')
plt.contour(X, Y, Z, levels=[0.1, 1, 10, 50, 100, 150, 200, 250, 300, 350, 400], colors='k', linewidths=1)

In [ ]:
guess = np.array([300., 450.])
delta = np.full(2, 200.0)

# ctr, min_chis, good_params, all_params = montecarlo.mcmin(egg_crate, guess, delta,
#                                                           print_status=True, detailed_output=True,
#                                                           red_factor=0.99, num_iters=10, num_pts=200)
# good_params, min_chis = metro(egg_crate, guess, delta, T0=100, cool_rate=0.5, num_iters=15, max_calls=10000)

good_params = [guess]
optres = minimize(egg_crate, guess, callback=good_params.append)
print(optres)
good_params = np.array(good_params)

# good_params = np.insert(good_params, 0, guess, axis=0)
# print(ctr)
# plt.plot(min_chis)

In [ ]:
X, Y = np.meshgrid(np.linspace(200, 512, 100), np.linspace(200, 512, 100))

XY = np.column_stack((X.ravel(), Y.ravel()))

Z = np.apply_along_axis(egg_crate, axis=1, arr=XY)
Z = Z.reshape(X.shape)

plt.plot(good_params[:, 0], good_params[:, 1], 'ro-')
plt.contour(X, Y, Z, levels=np.linspace(-1000, 1000, 10), colors='k', linewidths=1)
# plt.plot([512], [404.2319], 'co', markersize=20)

plt.gca().set_aspect(1)